In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from collections import Counter
import pickle


import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk import tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def clean_data(text):
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'\\n', ' ', text)
    text = re.sub(r'&[a-z]{0,7};', ' ', text)
    text = re.sub(r'\s{2,10}', ' ', text)
    text = re.sub(r'\s{2,10}', ' ', text)
    text = re.sub(r"\\'", r"'", text)
    text = re.sub(r'\\x\d{1,4}', '', text)
    return text

In [0]:
df = pd.read_csv('/content/drive/My Drive/dpl_dataset.csv')
fakes = df[df['label']=='fake']
reals = df[df['label']=='real']

reals.reset_index(drop=True, inplace=True)

# Sentence Level

In [6]:
# drop all that is less than 2 sentences
ids_fdrop = [ind  for ind, el in enumerate(list(fakes['text'])) if len(tokenize.sent_tokenize(el))<2]
ids_rdrop = [ind  for ind, el in  enumerate(list(reals['text'])) if len(tokenize.sent_tokenize(el))<2]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

6447

In [7]:
len(ids_fdrop), len(ids_rdrop)

(6444, 4)

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [9]:
fakes.shape, reals.shape

((273553, 3), (273553, 3))

In [10]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3  BY A.J. Miller, Jr.  The three men will make t...  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [11]:
fakes.tail(), reals.tail()

(                                                     text label     sampling
 273548  Governor Romney’s release of his final 2011 ta...  fake  temperature
 273549  Republican presidential candidate Mitt Romney ...  fake  temperature
 273550  Although it has been championed as a potential...  fake  temperature
 273551  It’s the season for stress. And this season wi...  fake  temperature
 273552  A prehistoric tooth with a cavity filled by be...  fake  temperature,
                                                      text label     sampling
 273548  Governor Romney’s release of his final 2011 ta...  real  No sampling
 273549  Republican presidential candidate Mitt Romney ...  real  No sampling
 273550  Although it has been championed as a potential...  real  No sampling
 273551  It’s the season for stress. Blame back-to-the-...  real  No sampling
 273552  A prehistoric tooth with a cavity filled by be...  real  No sampling)

In [0]:
# save only first two sentences

fakes['text'] = fakes['text'].apply(lambda x: ' '.join(tokenize.sent_tokenize(x)[:2]))
reals['text'] = reals['text'].apply(lambda x: ' '.join(tokenize.sent_tokenize(x)[:2]))

In [13]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3                                BY A.J. Miller, Jr.  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [14]:
ids_fdrop = [ind  for ind, el in enumerate(list(fakes['text'])) if len(tokenize.sent_tokenize(el))>2]
ids_rdrop = [ind  for ind, el in  enumerate(list(reals['text'])) if len(tokenize.sent_tokenize(el))>2]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

1144

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [16]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3                                BY A.J. Miller, Jr.  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [17]:
fakes.tail(), reals.tail()

(                                                     text label     sampling
 272404  Governor Romney’s release of his final 2011 ta...  fake  temperature
 272405  Republican presidential candidate Mitt Romney ...  fake  temperature
 272406  Although it has been championed as a potential...  fake  temperature
 272407  It’s the season for stress. And this season wi...  fake  temperature
 272408  A prehistoric tooth with a cavity filled by be...  fake  temperature,
                                                      text label     sampling
 272404  Governor Romney’s release of his final 2011 ta...  real  No sampling
 272405  Republican presidential candidate Mitt Romney ...  real  No sampling
 272406  Although it has been championed as a potential...  real  No sampling
 272407  It’s the season for stress. Blame back-to-the-...  real  No sampling
 272408  A prehistoric tooth with a cavity filled by be...  real  No sampling)

In [18]:
f = 0
for el in list(fakes['text']):
    if len(tokenize.sent_tokenize(el)) > 2:
        f +=1
r =0
for el in list(reals['text']):
    if len(tokenize.sent_tokenize(el)) > 2:
        r +=1
f, r

(0, 0)

# Drop duplicates

In [19]:
fakes.describe(), reals.describe()

(                                                     text   label     sampling
 count                                              272409  272409       272409
 unique                                             272087       1            4
 top     Before you go, we thought you'd like these... ...    fake  temperature
 freq                                                   66  272409       108133,
                                                      text   label     sampling
 count                                              272409  272409       272409
 unique                                             258627       1            1
 top     This post is in partnership with Time. The art...    real  No sampling
 freq                                                  139  272409       272409)

In [20]:
fakes.shape, reals.shape

((272409, 3), (272409, 3))

In [0]:
fakes_clean = fakes.drop_duplicates(inplace=False)
reals_clean = reals.drop_duplicates(inplace=False)

In [22]:
fakes_clean.shape, fakes.shape, reals_clean.shape, reals.shape

((272105, 3), (272409, 3), (258627, 3), (272409, 3))

In [23]:
ids_fdrop = [ind for ind in fakes.index if ind not in fakes_clean.index]
ids_rdrop = [ind for ind in reals.index if ind not in reals_clean.index]
ids_drop = list(set(ids_fdrop + ids_rdrop))
len(ids_drop)

13961

In [0]:
fakes.drop(np.array(ids_drop), inplace=True)
reals.drop(np.array(ids_drop), inplace=True)
fakes.reset_index(drop=True, inplace=True)
reals.reset_index(drop=True, inplace=True)

In [25]:
fakes.head(), reals.head()

(                                                text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  fake  temperature
 1  WASHINGTON, Dec. 23 - The National Security Ag...  fake  temperature
 2  IF outsized executive pay has indeed become a ...  fake  temperature
 3                                BY A.J. Miller, Jr.  fake  temperature
 4  Spinach has terrorized generations of veggie-p...  fake  temperature,
                                                 text label     sampling
 0  HAMBURG, Germany, June 3  As he left the socc...  real  No sampling
 1  WASHINGTON, Dec. 23 - The National Security Ag...  real  No sampling
 2  IF outsized executive pay has indeed become a ...  real  No sampling
 3  BY A.J. BENZA & MICHAEL LEWITTES If Simon Rex ...  real  No sampling
 4  Spinach has terrorized generations of veggie-p...  real  No sampling)

In [26]:
fakes.tail(), reals.tail()

(                                                     text label     sampling
 258443  Governor Romney’s release of his final 2011 ta...  fake  temperature
 258444  Republican presidential candidate Mitt Romney ...  fake  temperature
 258445  Although it has been championed as a potential...  fake  temperature
 258446  It’s the season for stress. And this season wi...  fake  temperature
 258447  A prehistoric tooth with a cavity filled by be...  fake  temperature,
                                                      text label     sampling
 258443  Governor Romney’s release of his final 2011 ta...  real  No sampling
 258444  Republican presidential candidate Mitt Romney ...  real  No sampling
 258445  Although it has been championed as a potential...  real  No sampling
 258446  It’s the season for stress. Blame back-to-the-...  real  No sampling
 258447  A prehistoric tooth with a cavity filled by be...  real  No sampling)

In [27]:
fakes.describe()

,text,label,sampling
count,258448,258448,258448
unique,258444,1,4
top,Republican Gov. Bobby Jindal (R-La.),fake,temperature
freq,2,258448,98408


In [28]:
reals.describe()

,text,label,sampling
count,258448,258448,258448
unique,258448,1,1
top,Electricity prices for Greater Boston resident...,real,No sampling
freq,1,258448,258448


# Counter

In [0]:
fakes_tokens = fakes['text'].apply(lambda x: x.split(' '))
reals_tokens = reals['text'].apply(lambda x: x.split(' '))

fake_split = Counter(x for xs in fakes_tokens for x in set(xs))
real_split = Counter(x for xs in reals_tokens for x in set(xs))

In [30]:
fake_split.most_common(20), real_split.most_common(20)

([('the', 211199),
  ('a', 174125),
  ('of', 165653),
  ('to', 161978),
  ('in', 149803),
  ('and', 146770),
  ('on', 91301),
  ('for', 89976),
  ('The', 88568),
  ('is', 86219),
  ('that', 84749),
  ('with', 66460),
  ('at', 64260),
  ('was', 57377),
  ('has', 57337),
  ('as', 51121),
  ('by', 49371),
  ('from', 48747),
  ('an', 48135),
  ('it', 42589)],
 [('the', 219852),
  ('a', 176626),
  ('of', 174583),
  ('to', 165496),
  ('and', 156775),
  ('in', 156152),
  ('on', 96010),
  ('for', 89757),
  ('is', 80363),
  ('that', 77858),
  ('The', 76510),
  ('with', 65523),
  ('at', 62086),
  ('has', 55358),
  ('was', 53371),
  ('as', 49431),
  ('from', 49031),
  ('by', 47992),
  ('an', 45679),
  ('his', 40588)])

# Save samplings datasets

In [44]:
samplings = fakes['sampling'].unique()

for sampling in samplings:
    
    fake_sampl = fakes[fakes['sampling']==sampling]
    real_sampl = reals.iloc[fake_sampl.index]
    assert real_sampl.shape == fake_sampl.shape, 'Different lenths of real and fake samplings'

    df_sampl = pd.concat([fake_sampl, real_sampl], axis=0)
    df_sampl = shuffle(df_sampl)
    df_sampl.reset_index(drop=True, inplace=True)
    
    df_sampl.to_csv('/content/drive/My Drive/data/sampling_{}.csv'.format(sampling), index=False)
    print('{} texts for {} sampling type were saved in sampling_{}.csv'.format(df_sampl.shape[0], sampling, sampling))

196816 texts for temperature sampling type were saved in sampling_temperature.csv
148052 texts for top_k sampling type were saved in sampling_top_k.csv
116960 texts for nucleus sampling type were saved in sampling_nucleus.csv
55068 texts for argmax sampling type were saved in sampling_argmax.csv


# Shuffle and save

In [0]:
df = pd.concat([fakes, reals], axis=0)
df.reset_index(drop=True, inplace=True)

In [0]:
df.head()

,text,label,sampling
0,"HAMBURG, Germany, June 3  As he left the socc...",fake,temperature
1,"WASHINGTON, Dec. 23 - The National Security Ag...",fake,temperature
2,IF outsized executive pay has indeed become a ...,fake,temperature
3,"BY A.J. Miller, Jr.",fake,temperature
4,Spinach has terrorized generations of veggie-p...,fake,temperature


In [0]:
df.tail()

,text,label,sampling
483197,"With a bit of hardware, your iPhone can become...",real,No sampling
483198,The Centers for Disease Control and Prevention...,real,No sampling
483199,The number of new HIV infections in the United...,real,No sampling
483200,AT&T says the throttling will only be temporar...,real,No sampling
483201,"While you're frying up some eggs and bacon, we...",real,No sampling


In [0]:
df = shuffle(df)
df.reset_index(drop=True, inplace=True)

In [0]:
df.head()

,text,label,sampling
0,"The police department in Green Mountain Falls,...",real,No sampling
1,"DHAKA, Bangladesh—Islamic State militants stor...",fake,nucleus
2,A few minutes into her visit with plastic surg...,real,No sampling
3,"Here is the second item from my ""Albany Inside...",real,No sampling
4,"Reversing a long and slow stock decline, share...",real,No sampling


In [0]:
df.tail()

,text,label,sampling
483197,"Before you go, we thought you'd like these... ...",real,No sampling
483198,(CNN) -- When Egypt's first democratically ele...,fake,top_k
483199,These includes incentivising German women to h...,real,No sampling
483200,Worshippers in ponchos of all colors held up f...,fake,top_k
483201,Serena Williams just won her seventh Wimbledon...,real,No sampling


In [0]:
fake = list(fakes['text'])
real = list(reals['text'])

df.to_csv('/content/drive/My Drive/postpoc_dpl_dataset.csv', index=False)

In [0]:
with open('/content/drive/My Drive/fake', 'wb') as fp:
    pickle.dump(fake, fp)

with open('/content/drive/My Drive/real', 'wb') as fp:
    pickle.dump(real, fp)

In [0]:
# with open ('fake', 'rb') as fp:
#     itemlist = pickle.load(fp)